In [1]:
from ctapipe.io import EventSource
from ctapipe import utils
from ctapipe.visualization import CameraDisplay
from ctapipe.calib import CameraCalibrator
from ctapipe.io import DataWriter
from ctapipe.io import HDF5TableWriter
import numpy as np
from ctapipe.image import hillas_parameters, tailcuts_clean
from ctapipe_funcs import open_simtel

/home/vhep/mescob11/.conda/envs/ctapipe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Opening simulation file with ctapipe classes
path = "simtel_files/gamma_20deg_0deg_run19___cta-prod3-sct_desert-2150m--subarray-1-sct.simtel.gz"
source, calibrator = open_simtel(path, back_seeking=True, calibration=True)

In [5]:
# 2 pass cleaning and writing hillas parameters to hdf5 table  

with HDF5TableWriter(filename='hillas_param_and_clean_image.h5', 
                     group_name='dl1', overwrite=True) as writer:       # hdf5 class writer
    
    for event in source:                                                # looping over triggered events in simulation file
        calibrator(event)                                               # calibrator class creates dl0 dl1 data

        for tel_id, tel_data in event.dl1.tel.items():                  # accessing dl1 data for every triggered event                           
            tel = source.subarray.tel[tel_id]                           # telescope type
            mask = tailcuts_clean(tel.camera.geometry, tel_data.image)  # 2 pass cleaning function with default values
                                                                        # mask is an array of True/False indicating if pixel passed cleaning
                                                                        
            if np.count_nonzero(mask) > 0:                              # makes sure there are clean pixels otherwise hillas fitting fails
                params = hillas_parameters(tel.camera.geometry[mask],   # hillas fitting function
                                           tel_data.image[mask])
                print(params['intensity'], event.simulation.shower.energy, np.count_nonzero(mask))
                #writer.write("hillas", params)                          # writes hillas parameters for each telescope with a valid image to
                                                                        # to hdf5 table 
                                                                        # Note images are not included in table

/home/vhep/mescob11/.conda/envs/ctapipe/lib/python3.11/site-packages/ctapipe/io/simteleventsource.py:702: UserWarning: Backseeking to start of file.
  warnings.warn("Backseeking to start of file.")


50.171093463897705 0.15292209386825562 TeV 7
152.72081422805786 0.3752979040145874 TeV 16
180.09809684753418 0.3752979040145874 TeV 18
144.4182620048523 0.3752979040145874 TeV 11
173.7878394126892 0.3752979040145874 TeV 13
71.38175678253174 0.3752979040145874 TeV 8
142.0355830192566 0.3752979040145874 TeV 13
240.23351287841797 0.3752979040145874 TeV 19
80.70024108886719 0.3752979040145874 TeV 8
234.44278860092163 0.3752979040145874 TeV 16
181.77296543121338 0.3752979040145874 TeV 17
44.52316951751709 0.3752979040145874 TeV 5
62.563055992126465 0.3752979040145874 TeV 6
48.43528509140015 0.3752979040145874 TeV 5
27.721944332122803 0.12565098702907562 TeV 4
108.2314887046814 0.9595512747764587 TeV 12
235.92376565933228 0.9595512747764587 TeV 21
49.4663200378418 0.9595512747764587 TeV 7
439.4531650543213 0.9595512747764587 TeV 27
629.6740674972534 0.9595512747764587 TeV 49
63.35760545730591 0.9595512747764587 TeV 7
23.801400661468506 0.9595512747764587 TeV 3
568.1274018287659 0.95955127477

In [3]:
# writing dl1 image to hdf5 table

with HDF5TableWriter(filename='hdf5_files/images_no_cleaning.h5', group_name='dl1', overwrite=True) as writer:
    for event in source:
        calibrator(event)

        for tel_id, tel_data in event.dl1.tel.items(): 
            writer.write("images_no_cleaning", tel_data)     # Full image is included in table

Column image_mask with value None of type <class 'NoneType'>  of container DL1CameraContainer in table images_no_cleaning not writable, skipping
Column parameters with value None of type <class 'NoneType'>  of container DL1CameraContainer in table images_no_cleaning not writable, skipping


In [14]:
# writing clean dl1 image to hdf5 table
# pixels that do not pass cleaning are set to zero

with HDF5TableWriter(filename='hdf5_test_files/images_2pass_cleaning.h5', group_name='dl1', overwrite=True) as writer:
    for event in source:
        calibrator(event)

        for tel_id, tel_data in event.dl1.tel.items():                                            
            tel = source.subarray.tel[tel_id]
            cleaned = tel_data.image.copy()                                 
            mask = tailcuts_clean(tel.camera.geometry, tel_data.image)
            tel_data.image[~mask] = 0
            tel_data.peak_time[~mask] = 0                     # mask allows to set pixels (counts and time) to zero
            writer.write('images_cleaned',tel_data)           # cleaned images written to h5 table

/home/vhep/programs/anaconda3/envs/ctlearn060/lib/python3.8/site-packages/ctapipe/io/simteleventsource.py:373: UserWarning: Backseeking to start of file.
  warnings.warn("Backseeking to start of file.")
Column image_mask of container DL1CameraContainer in table images_cleaned not writable, skipping
Column parameters of container DL1CameraContainer in table images_cleaned not writable, skipping


In [5]:
# writing data reduced dl1 images with ctapipe method to hdf5 table 
# ctapipe already has a method that does a more conservative reduction for us
# utilizing a 2 pass cleaning method. It specifically calls tail_cuts and dilation
# modules to do the process.
new_source , data_red_calibrator = open_simtel(path, back_seeking=True, calibration=True, data_reducer=True)

with HDF5TableWriter(filename='hdf5_test_files/images_ctapipe_data_reduced.h5', group_name='dl1', overwrite=True) as writer:
    for event in new_source:
        data_red_calibrator(event)

        for tel_id, tel_data in event.dl1.tel.items(): 
            writer.write("images_data_reduced", tel_data)     # data reduced image is included in table

TypeError: 'str' object is not callable

In [14]:
# Data reducer doesn't seem to work for the time being
source = EventSource(path)
#source = EventSource("simtel_files/gamma_20deg_0deg_run19___cta-prod3-sct_desert-2150m--subarray-1-sct.simtel.gz")
subarray = source.subarray
calib = CameraCalibrator(subarray, data_volume_reducer="TailCutsDataVolumeReducer")
calib
#for event in source:
#    calib(event)

apply_peak_time_shift,"[('type', '*', True)]","Apply peak time shift corrections. Apply the remaining absolute and fractional time shift corrections to the peak time after pulse extraction. If `apply_waveform_time_shift` is False, this will apply the full time shift (default: traitlets.Undefined)"
apply_waveform_time_shift,"[('type', '*', False)]",Apply waveform time shift corrections. The minimal integer shift to synchronize waveforms is applied before peak extraction if this option is True (default: traitlets.Undefined)
data_volume_reducer_type,NullDataVolumeReducer,DataVolumeReducer to use. (default: NullDataVolumeReducer)
image_extractor_type,"[('type', '*', 'NeighborPeakWindowSum')]",Name of the ImageExtractor subclass to be used. (default: traitlets.Undefined)


In [17]:
# writing dl0 event container to hdf5 table

with HDF5TableWriter(filename='hdf5_test_files/dl0_event_container.h5', group_name='dl0', overwrite=True) as writer:
    for event in source:
        calibrator(event)

        for tel_id, tel_data in event.dl0.tel.items(): 
            writer.write("dl0_images", tel_data)            # Full dl0 telescope image data is included in table

/home/vhep/programs/anaconda3/envs/ctlearn060/lib/python3.8/site-packages/ctapipe/io/simteleventsource.py:373: UserWarning: Backseeking to start of file.
  warnings.warn("Backseeking to start of file.")
